![](https://storage.googleapis.com/kaggle-competitions/kaggle/23249/logos/header.png)

# Motivation

Motivation: I wanted to implement a notebook where it would be easy enough to run as just editing the config (first cell) and changing models (if you want to) in order to save experiment time. Another major point for me was being able to run it on both the GPU and TPU.

The following ideas are implemented in this Notebook:
* Optimizers : Adam, AdamW, AdamP, Ranger
* Schedulers : ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts', 'OneCycleLR' ,'GradualWarmupSchedulerV2']
* Models    : ['deit_base_patch16_384','vit_large_patch16_384','tf_efficientnet_b4_ns','resnext50_32x4d'] it also works for different effnets and resnexts
* Loss Fns   : ['BCEwithLogits']
* Augmentations: HeavyAug+Cutout(Train),LightAug (only rotations), HeavyAug, AutoAugment(imagenet)

Any feedback is appreciated!
Be sure to also checkout:
* Inference: tbd

# Config

In [ ]:
# ====================================================
# Configurations
# ====================================================
import os
class CFG:
    DEBUG = False
    
    #Model Params
    device = 'GPU' #['CPU','GPU','TPU']
    N_FOLDS = 5
    MODEL_NAME = 'tf_efficientnet_b4_ns' # Recommended : ['tf_efficientnet_b4_ns','resnext50_32x4d']
    pretrained = True   
    EPOCHS = 10 if not DEBUG else 3 # more is definitely plausible
    TRAIN_FOLDS = [0] if DEBUG else [i for i in range(N_FOLDS)] #Folds to be Trained
    N_CLASSES = 1
    in_channels = 1
    
    scheduler_name = 'CosineAnnealingWarmRestarts'
    # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts', 'OneCycleLR', 'GradualWarmupSchedulerV2']
    scheduler_update = 'epoch' #['batch','epoch']
    criterion_name = 'BCEWithLogitsLoss'
    # ['BCEWithLogitsLoss','ROC-STAR']
    optimizer_name = 'AdamP' #['Adam','AdamW','AdamP','Ranger'] -> AdamP doesn't work on TPUs
    LR_RAMPUP_EPOCHS = 1
    LR_SUSTAIN_EPOCHS = 0
    
    FREEZE = False #If you fine tune after START_FREEZE epochs
    START_FREEZE = 8
    
    #Training Params
    BATCH_SIZE = 1024
    
    LR = 2e-4
    LR_START =1e-4
    LR_MIN = 8e-7
    weight_decay = 0
    eps = 1e-8
    PATIENCE = 1
      
    #CosineAnnealingWarmRestarts
    T_0 = EPOCHS
    
    #CosineAnnealingLR
    T_max = EPOCHS
    
    NUM_WORKERS = 4
    
    model_print = False #If the model architecture is printed
    tqdm = True #If training bar is shown
    
    #n_procs = number of replicas -> TPU
    n_procs = 8 #You can set it to 1 and run a TPU as a GPU if you want
    SEED = 42
    saved_models = {}

# Installations

In [ ]:
# ====================================================
# Required Installations
# ====================================================
!pip install timm

if CFG.device == 'TPU':
    import os
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
    os.system('export XLA_USE_BF16=1')
    os.system('export XLA_TENSOR_ALLOCATOR_MAXSIZE=100000000')
    import torch_xla.core.xla_model as xm
    import torch_xla.distributed.parallel_loader as pl
    import torch_xla.distributed.xla_multiprocessing as xmp
    import ignite.distributed as idist
    #CFG.LR = CFG.LR * CFG.n_procs
    CFG.BATCH_SIZE = CFG.BATCH_SIZE / CFG.n_procs
    
if CFG.optimizer_name == 'Ranger':
    !pip install --quiet '../input/pytorch-ranger'
elif CFG.optimizer_name == 'AdamP':
    !pip install adamp

# Library

In [ ]:
# ====================================================
# Library
# ====================================================

import random
import math
import time

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import numpy as np
import cv2
from sklearn.model_selection import GroupKFold, StratifiedKFold,KFold
from sklearn.metrics import accuracy_score,roc_auc_score

import torch
import torch.nn as nn
import torchvision
from torchvision import models as tvmodels
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
from tqdm import tqdm
import timm



import albumentations as A
from albumentations import Compose
from albumentations.pytorch import ToTensorV2
import seaborn as sns



from PIL import Image, ImageOps, ImageEnhance, ImageChops


if CFG.optimizer_name == 'Ranger':
    from pytorch_ranger import Ranger
elif CFG.optimizer_name == 'AdamP':
    from adamp import AdamP
    
if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
    from warmup_scheduler import GradualWarmupScheduler


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
import librosa

# Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
# credit to https://www.kaggle.com/thedrcat/g2net-fastai-resnet34-starter-mel
def id2path(id, is_test):
    a, b, c = id[0], id[1], id[2]
    if is_test: return f'../input/g2net-gravitational-wave-detection/test/{a}/{b}/{c}/{id}.npy'
    return f'../input/g2net-gravitational-wave-detection/train/{a}/{b}/{c}/{id}.npy'

def get_score(y_true, y_pred):
    score = roc_auc_score(y_true, y_pred)
    return score

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
    class GradualWarmupSchedulerV2(GradualWarmupScheduler):
        def __init__(self, optimizer = None, multiplier = CFG.LR/CFG.LR_START, total_epoch = CFG.LR_RAMPUP_EPOCHS, after_scheduler=None):
            super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
            self.after_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = CFG.T_0 - CFG.LR_RAMPUP_EPOCHS, T_mult=1, eta_min=CFG.LR_MIN, last_epoch=-1)
        def get_lr(self):
            if self.last_epoch > self.total_epoch:
                if self.after_scheduler:
                    if not self.finished:
                        self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                        self.finished = True
                    return self.after_scheduler.get_lr()
                return [base_lr * self.multiplier for base_lr in self.base_lrs]
            if self.multiplier == 1.0:
                return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
            else:
                return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
  
    
def GetScheduler(scheduler_name,optimizer,batches):
    #['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts', 'OneCycleLR', 'GradualWarmupSchedulerV2']
    if scheduler_name == 'OneCycleLR':
        return torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr = 1e-2,epochs = CFG.EPOCHS,steps_per_epoch = batches+1,pct_start = 0.1)
    elif scheduler_name == 'CosineAnnealingWarmRestarts':
        return torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = CFG.T_0, T_mult=1, eta_min=CFG.LR_MIN, last_epoch=-1)
    elif scheduler_name == 'CosineAnnealingLR':
        return torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = CFG.T_max, eta_min=0, last_epoch=-1)
    elif scheduler_name == 'ReduceLROnPlateau':
        return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor=0.1, patience=1, threshold=0.0001, cooldown=0, min_lr=CFG.LR_MIN, eps=CFG.eps)
    elif scheduler_name == 'GradualWarmupSchedulerV2':
        return GradualWarmupSchedulerV2(optimizer=optimizer)
    
def GetOptimizer(optimizer_name,parameters):
    #['Adam','Ranger']
    if optimizer_name == 'Adam':
        if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
            return torch.optim.Adam(parameters, lr=CFG.LR_START, weight_decay=CFG.weight_decay, amsgrad=False)
        else:
            return torch.optim.Adam(parameters, lr=CFG.LR, weight_decay=CFG.weight_decay, amsgrad=False)
    elif optimizer_name == 'AdamW':
        if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
            return torch.optim.AdamW(parameters, lr=CFG.LR_START, weight_decay=CFG.weight_decay, amsgrad=False)
        else:
            return torch.optim.Adam(parameters, lr=CFG.LR, weight_decay=CFG.weight_decay, amsgrad=False)
    elif optimizer_name == 'AdamP':
        if CFG.scheduler_name == 'GradualWarmupSchedulerV2':
            return AdamP(parameters, lr=CFG.LR_START, weight_decay=CFG.weight_decay)
        else:
            return AdamP(parameters, lr=CFG.LR, weight_decay=CFG.weight_decay)
    elif optimizer_name == 'Ranger':
        return Ranger(parameters,lr = CFG.LR,alpha = 0.5, k = 6,N_sma_threshhold = 5,betas = (0.95,0.999),eps=CFG.eps,weight_decay=CFG.weight_decay)

def print_scheduler(scheduler = None,scheduler_update = CFG.scheduler_update,optimizer = None, batches = -1, epochs = -1, model = None):
    lrs = []
    if scheduler_update == 'epoch':
        for epoch in range(epochs):
            scheduler.step(epoch)
            lrs.append(optimizer.param_groups[0]["lr"])
        plt.figure(figsize=(15,4))
        plt.plot(lrs)
    elif scheduler_update == 'batch':
        for epoch in range(epochs):
            for batch in range(batches):
                scheduler.step()
                lrs.append(optimizer.param_groups[0]["lr"])
        plt.figure(figsize=(15,4))
        plt.plot(lrs)
    
SEED = CFG.SEED
seed_everything(SEED)  
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Transforms

In [ ]:
# ====================================================
# Transformations
# ====================================================
# credit to https://www.kaggle.com/thedrcat/g2net-fastai-resnet34-starter-mel
def sample_to_mel(_id, is_test):
    x = np.load(id2path(_id, is_test))
    spectrogram = []
    for i in range(3):
        mel = librosa.feature.melspectrogram(x[i]/x[i].max(), 
                                                sr=2048,
                                                n_mels=16,
                                               hop_length=16)
        mel = mel[:,:256]
        mel = librosa.power_to_db(mel).astype(np.float32)
        mel = mel.reshape(64,64)
        spectrogram.append(mel)
    spectrogram = np.stack(spectrogram)
    return spectrogram

# Datasets

In [ ]:
# ====================================================
# Datasets
# ====================================================
def retrieve_df(df,name,idx):
    series = df[name].iloc[idx]
    series.reset_index(drop=True,inplace=True)
    
    return series

class G2NetDataset(torch.utils.data.Dataset):
    def __init__(self, features, target, is_test=False,transformed = True,file_names = []):
        self.features,self.target,self.is_test,self.transformed,self.file_names = features,target,is_test,transformed,file_names
        
    def __getitem__(self, i):
        if not self.transformed:
            img = self.features.loc[i]
            img = sample_to_mel(img, self.is_test) if self.transform else img
            if self.is_test:
                return (torch.tensor(img, dtype=torch.float), torch.tensor(0, dtype=torch.float))
            else:
                tgt = self.target.loc[i]
                return (torch.tensor(img, dtype=torch.float), torch.tensor(tgt, dtype=torch.float))
        else:
            #adapted from https://www.kaggle.com/yasufuminakama/g2net-efficientnet-b0-baseline-training
            file_path = self.file_names[i]
            image = np.load(file_path)
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
            tgt = self.target.loc[i]
            return (image, torch.tensor(tgt, dtype=torch.float))
    
    def __len__(self): return len(self.target)
    

# CV Split

In [ ]:
# ====================================================
# CV Split
# ====================================================
#adapted from https://www.kaggle.com/yasufuminakama/g2net-efficientnet-b0-baseline-training
train_df = pd.read_csv("../input/g2net-gravitational-wave-detection/training_labels.csv")
test_df = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

def get_train_file_path(image_id):
    return "../input/g2net-n-mels-128-train-images/{}.npy".format(image_id)

def get_test_file_path(image_id):
    return "../input/g2net-n-mels-128-test-images/{}.npy".format(image_id)

train_df['file_path'] = train_df['id'].apply(get_train_file_path)
test_df['file_path'] = test_df['id'].apply(get_test_file_path)

#modified from https://www.kaggle.com/thedrcat/g2net-fastai-resnet34-starter-mel
if CFG.DEBUG:
    train_df = train_df.sample(frac=0.01).reset_index(drop=True)
skf = StratifiedKFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)
skf.get_n_splits(np.arange(train_df.shape[0]), train_df['target'])
folds = [(idxT,idxV) for i,(idxT,idxV) in enumerate(skf.split(np.arange(train_df.shape[0]), train_df['target']))]
    
sns.countplot(data=train_df, x="target")

# Model

In [ ]:
# ====================================================
# Model
# ====================================================
class G2Net(nn.Module):
    def __init__(self, model_name=CFG.MODEL_NAME, pretrained=CFG.pretrained,in_chans = CFG.in_channels):
        super().__init__()
        self.model_name = model_name
        if model_name == 'deit_base_patch16_224' or model_name == 'deit_base_patch16_384':
            self.model = torch.hub.load('facebookresearch/deit:main', model_name, pretrained=pretrained, in_chans=in_chans)
        else:
            self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=in_chans)
        if 'efficientnet' in model_name:
            self.n_features = self.model.classifier.in_features
            self.model.classifier = nn.Linear(self.n_features, CFG.N_CLASSES)
        elif model_name == 'vit_large_patch16_384' or model_name == 'deit_base_patch16_224' or model_name == 'deit_base_patch16_384':
            self.n_features = self.model.head.in_features
            self.model.head = nn.Linear(self.n_features, CFG.N_CLASSES)
        elif 'resnext' in model_name:
            self.n_features = self.model.fc.in_features
            self.model.fc = nn.Linear(self.n_features, CFG.N_CLASSES)
        
    def forward(self, x):
        return self.model(x)
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.model.parameters():
            param.requires_grad = False
            
        if 'efficientnet' in self.model_name:
            for param in self.model.classifier.parameters():
                param.requires_grad = True
        elif self.model_name == 'vit_large_patch16_384' or 'deit_base_patch16_224':
            for param in self.model.head.parameters():
                param.requires_grad = True
        elif 'resnext' in self.model_name:
            for param in self.model.fc.parameters():
                param.requires_grad = True
            
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.model.parameters():
            param.requires_grad = True
model = G2Net()

# Training Loop

In [ ]:
# ====================================================
# Training Loop
# ====================================================
def train_one_epoch(model,optimizer,scheduler,scaler,train_loader,criterion,batches,epoch,DEVICE):   
    tr_loss = 0.0
    scores = 0.0
    trn_epoch_result = dict()
    model.train()
    if CFG.tqdm:
        progress = tqdm(enumerate(train_loader), desc="Loss: ", total=len(train_loader))
    else:
        progress = enumerate(train_loader)
    for i, (images,labels) in progress:
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        optimizer.zero_grad()
        
        if CFG.device == 'TPU':
            logits = model(images).reshape((-1))
            loss = criterion(logits, labels)
            loss.backward()
            xm.optimizer_step(optimizer)
        else:
            with autocast():
                logits = model(images).reshape((-1))
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

        
        if CFG.scheduler_update == 'batch':
            if not CFG.scheduler_name == 'OneCycleLR':
                scheduler.step(epoch + i/len(train_loader))
            else:
                scheduler.step()

        tr_loss += loss.detach().item()
        
        if CFG.tqdm:
            trn_epoch_result['Epoch'] = epoch
            trn_epoch_result['train_loss'] = round(tr_loss/(i+1), 4)
            trn_epoch_result['LR'] = round(optimizer.param_groups[0]["lr"],7)

            progress.set_description(str(trn_epoch_result))
        else:
            print(tr_loss/(i+1))
    if CFG.scheduler_update == 'epoch':
            scheduler.step(epoch+1)
        
def val_one_epoch(model,DEVICE,loader,val_criterion,epoch,get_output = False):
    val_loss = 0.0
    scores = 0.0
    model.eval()
    val_progress = tqdm(enumerate(loader), desc="Loss: ", total=len(loader))
    with torch.no_grad():
        for i, (images,labels) in val_progress:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            logits = model(images).reshape((-1))
            val_loss_value = val_criterion(logits,labels)
            val_loss += val_loss_value.detach().item()

            scores += get_score(labels.to('cpu').numpy(),logits.sigmoid().to('cpu').numpy())

            val_epoch_result = dict()
            val_epoch_result['Epoch'] = epoch
            val_epoch_result['val_loss'] = round(val_loss/(i+1), 4)

            val_epoch_result['val_acc'] = round(scores/(i+1), 4)
            val_progress.set_description(str(val_epoch_result))
    if get_output:
        return val_loss/len(loader),scores/len(loader)
        
def model_train():
    if CFG.device == 'TPU':
        DEVICE = xm.xla_device()
    else:
        DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
    for fold,(idxT, idxV) in enumerate(folds):
        if fold not in CFG.TRAIN_FOLDS:
            continue
        #if xm.is_master_ordinal():
        #    xm.master_print(fold)
        #______INSTANTIATE TRAINING DATASETS_____
        x_train = retrieve_df(train_df,'id',idxT)
        y_train = retrieve_df(train_df,'target',idxT)
        x_val = retrieve_df(train_df,'id',idxV)
        y_val = retrieve_df(train_df,'target',idxV)
        train_set = G2NetDataset(x_train,y_train, is_test=False,transformed = True,file_names = retrieve_df(train_df,'file_path',idxT))
        val_set = G2NetDataset(x_val,y_val, is_test=False,transformed = True,file_names = retrieve_df(train_df,'file_path',idxV))
        if CFG.device == 'TPU':
            train_sampler = torch.utils.data.distributed.DistributedSampler(
                train_set,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=True)
            train_loader = DataLoader(train_set, batch_size=CFG.BATCH_SIZE, sampler=train_sampler,drop_last=True, num_workers=CFG.NUM_WORKERS)
        
            val_sampler = torch.utils.data.distributed.DistributedSampler(
                val_set,
                num_replicas=xm.xrt_world_size(),
                rank=xm.get_ordinal(),
                shuffle=False)
            val_loader = DataLoader(val_set, batch_size=CFG.BATCH_SIZE, sampler=val_sampler,drop_last=True, num_workers=CFG.NUM_WORKERS)
            scaler = None
        else:
            print(f"Start of Fold {fold}")
            train_loader = DataLoader(train_set, batch_size=CFG.BATCH_SIZE, shuffle=True,drop_last=True, num_workers=CFG.NUM_WORKERS,pin_memory = True)
            val_loader = DataLoader(val_set, batch_size=CFG.BATCH_SIZE, shuffle=False,drop_last=True, num_workers=CFG.NUM_WORKERS,pin_memory = True)
            scaler = GradScaler()
            
        batches = len(train_loader)
        val_batches = len(val_loader)

        #INSTANTIATE FOLD MODEL
        if CFG.model is None:
            if CFG.device == 'TPU':
                if xm.is_master_ordinal(local=True):
                    CFG.model = G2Net(model_name=CFG.MODEL_NAME, pretrained=True)
                xm.rendezvous('ModelDone')
            else:
                CFG.model = G2Net(model_name=CFG.MODEL_NAME, pretrained=True)
        model = CFG.model.to(DEVICE)

        criterion = CFG.criterion.to(DEVICE)
        val_criterion = CFG.val_criterion.to(DEVICE)

        optimizer = GetOptimizer(CFG.optimizer_name, model.parameters())
        scheduler = GetScheduler(CFG.scheduler_name, optimizer,batches)
        
        saved_model = None
        best_val_acc = 0.0
        best_val_loss = 1e3
        fold_patience = 0.0
        for epoch in range(CFG.EPOCHS):
            if epoch >= CFG.START_FREEZE and CFG.FREEZE:
                print('Model Frozen -> Train Classifier Only')
                info = torch.load(saved_model,map_location = torch.device(DEVICE))
                model.load_state_dict(info)
                model.freeze()
                
                CFG.FREEZE = False
            #______TRAINING______
            if CFG.device == 'TPU':
                para_train_loader = pl.ParallelLoader(train_loader, [DEVICE])
                train_one_epoch(model,optimizer,scheduler,scaler,para_train_loader.per_device_loader(DEVICE),criterion,batches,epoch,DEVICE)
                del para_train_loader
                gc.collect()
            else:
                train_one_epoch(model,optimizer,scheduler,scaler,train_loader,criterion,batches,epoch,DEVICE)
            
            #______VALIDATION_______
            if CFG.device == 'TPU':
                para_val_loader = pl.ParallelLoader(val_loader, [DEVICE])
                val_loss, val_acc = val_one_epoch(model,DEVICE,para_val_loader.per_device_loader(DEVICE),val_criterion,epoch,get_output = True)
                del para_val_loader
                gc.collect()
                val_loss = np.sum(idist.all_gather(torch.tensor(val_loss)).to('cpu').numpy())/CFG.n_procs
                val_acc = np.sum(idist.all_gather(torch.tensor(val_acc)).to('cpu').numpy())/CFG.n_procs
                xm.master_print(f'Fold Ended at {round(val_acc, 4)} val accuracy')
            else:
                val_loss, val_acc = val_one_epoch(model,DEVICE,val_loader,val_criterion,epoch,get_output = True)
            
            if val_acc > best_val_acc:
                fold_patience = 0
                best_val_loss = val_loss/val_batches
                best_val_acc = val_acc
                if CFG.device == 'TPU':
                    xm.save(CFG.model.state_dict(),
                                f'{CFG.MODEL_NAME}_f{fold}_TPU_b{round(best_val_acc, 4)}.pth')
                    if saved_model is not None:
                        try:
                            os.remove("./"+saved_model)
                        except:
                            a = 1
                    xm.master_print(f'Model Saved at {round(best_val_acc, 5)} accuracy')
                    saved_model = f'{CFG.MODEL_NAME}_f{fold}_TPU_b{round(best_val_acc, 4)}.pth'
                    CFG.saved_models[fold] = round(best_val_acc, 4)
                else:
                    torch.save(model.state_dict(),
                            f'{CFG.MODEL_NAME}_f{fold}_b{round(best_val_acc, 4)}.pth')
                    if saved_model is not None:
                        try:
                            os.remove("./"+saved_model)
                        except:
                            a = 1
                    saved_model = f'{CFG.MODEL_NAME}_f{fold}_b{round(best_val_acc, 4)}.pth'
                    CFG.saved_models[fold] = round(best_val_acc, 4)
                    print(f'Model Saved at {round(best_val_acc, 5)} accuracy')
            else:
                fold_patience += 1
                if fold_patience >= CFG.PATIENCE:
                    print(f'Early stopping due to model not improving for {CFG.PATIENCE} epochs')
                    CFG.model = None
                    break
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        CFG.model = None
                
def _map_fn(index,flags):
    torch.set_default_tensor_type('torch.FloatTensor')
    a = model_train()

# Run

In [ ]:
CFG.model = model
CFG.criterion = nn.BCEWithLogitsLoss()
CFG.val_criterion = nn.BCEWithLogitsLoss()
if CFG.device == 'TPU':
    FLAGS = {}
    xmp.spawn(_map_fn, args=(FLAGS,), nprocs=CFG.n_procs, start_method='fork')
else:
    torch.set_default_tensor_type('torch.FloatTensor')
    a = model_train()

# Inference - Ensemble

In [ ]:
# ====================================================
# Inference
# ====================================================
test_df['file_path'] = test_df['id'].apply(get_test_file_path)
x_test = test_df['id']
y_test = test_df['target']
folds_preds = np.zeros((len(CFG.TRAIN_FOLDS),len(test_df)))
for fold in CFG.TRAIN_FOLDS:
    if CFG.device == 'TPU':
        info = torch.load(f'{CFG.MODEL_NAME}_f{fold}_TPU_b{CFG.saved_models[fold]}.pth',map_location = torch.device(DEVICE))
    else:
        info = torch.load(f'{CFG.MODEL_NAME}_f{fold}_b{CFG.saved_models[fold]}.pth',map_location = torch.device(DEVICE))
    model.load_state_dict(info)
    test_set = G2NetDataset(x_test,y_test, is_test=True,transformed = True,file_names = test_df['file_path'])
    test_loader = DataLoader(test_set, batch_size=CFG.BATCH_SIZE, shuffle=False,drop_last=False, num_workers=CFG.NUM_WORKERS,pin_memory = True)
    test_progress = tqdm(enumerate(test_loader), desc="Loss: ", total=len(test_loader))
    model.eval()
    preds_arr = []
    with torch.no_grad():
        for i, (images,labels) in test_progress:
            images = images.to(DEVICE)

            logits = model(images).reshape((-1))
            preds = logits.sigmoid().to('cpu').numpy()
            preds_arr.append(preds)

    folds_preds[fold,:] = np.concatenate(preds_arr)
folds_preds = np.mean(folds_preds,axis = 0)
test_df['target'] = folds_preds
test_df.drop('file_path',axis = 1,inplace=True)
test_df.to_csv('output.csv', index=False)